---
# Create ordered list of SOILED netcdf output from monte-carlo 
# Identify missing/incomplete runs
---

### The monte carlo runs were completed on Compute Canada's supercomputer, `Graham`.  This code was developed to be used on that system.  It requires initialization of a Virtual Environment 

In [1]:
from glob import glob
import os
import sys
import pandas
sys.path.insert(
    0, 
    '/home/rmueller/projects/def-allen/rmueller/MIDOSS/Visualization/scripts'
) 
from aggregate_SOILED_tools import get_MOHID_netcdf_filenames
results_dir='/scratch/dlatorne/MIDOSS/runs/monte-carlo'
output_dir ='/scratch/rmueller/MIDOSS/Results'

In [2]:
files = get_MOHID_netcdf_filenames(results_dir, output_dir)

In [3]:
oil_types = [
    'akns', 
    'bunker', 
    'dilbit', 
    'jet', 
    'diesel', 
    'gas', 
    'other'
]
total = 0
time_per_file = 0.575
minutes_to_hours = 1/60
for oil in oil_types:
    number_of_files = len(files[oil])
    time_to_complete = time_per_file * number_of_files * minutes_to_hours
    print(f'{oil}: {number_of_files} runs, approx {time_to_complete:.2f} hours to complete')
    total+=len(files[oil])
print(f'TOTAL RUNS: {total}')

akns: 64 runs, approx 0.61 hours to complete
bunker: 3323 runs, approx 31.85 hours to complete
dilbit: 1 runs, approx 0.01 hours to complete
jet: 26 runs, approx 0.25 hours to complete
diesel: 6192 runs, approx 59.34 hours to complete
gas: 73 runs, approx 0.70 hours to complete
other: 68 runs, approx 0.65 hours to complete
TOTAL RUNS: 9747


### Find missing files

In [4]:
# get list of runsets
runsets = sorted(glob(os.path.join(results_dir,"near-BP_*")))
finished = pandas.DataFrame({'filenames':files['all']})

In [ ]:
# get list of runsets
runsets = sorted(glob(os.path.join(results_dir,"near-BP_*")))
finished = pandas.DataFrame({'filenames':files['all']})

In [46]:
list_of_incomplete = []
n_missing = 0
for runset in runsets: 
    finished_runset = finished[finished['filenames'].str.contains(runset)]
    nruns = f'{run}'.split('-')[3:][0].split('_')[0]
    nruns_finished = len(finished_runset)
    if int(nruns)!=nruns_finished:
        if nruns_finished>0:
            print(f'{run}'.split('/')[-1],f': {nruns_finished} of {nruns}')
            list_of_incomplete.append(f'{run}'.split('/')[-1])
            n_missing+=int(nruns)-nruns_finished


near-BP_2000th-100_2021-10-02T144815 : 76 of 100
near-BP_2004th-100_2021-10-02T144852 : 95 of 100
near-BP_46th-100_2021-10-02T142422 : 96 of 100
near-BP_49th-100_2021-10-03T140133 : 86 of 100
near-BP_4th-100_2021-09-24T125848 : 96 of 100
near-BP_53th-100_2021-10-03T140217 : 80 of 100
near-BP_55th-100_2021-10-03T140235 : 99 of 100
near-BP_79th-100_2021-10-07T121854 : 90 of 100
near-BP_88th-100_2021-10-11T190627 : 98 of 100
near-BP_94th-103_2021-10-13T133218 : 101 of 103


In [115]:
missing_runs={}
completed_runs={}
for runset in list_of_incomplete: 
    completed_runs[runset]=[]
    finished_runset = finished[finished['filenames'].str.contains(runset)]
    for run in finished_runset['filenames']:
        completed_runs[runset].append(int(run.split('.')[0].split('-')[-1]))
    completed_runs[runset].sort()
    for i in range(len(completed_runs[runset]) - 1):
        #print(completed_runs[runset][i],completed_runs[runset][i+1])
        if (completed_runs[runset][i+1] - completed_runs[runset][i])>1:
            run_list = numpy.arange(
                completed_runs[runset][i]+1,completed_runs[runset][i+1]
            )
            if runset in missing_runs:
                missing_runs[runset]=numpy.append(missing_runs[runset],run_list)
            else:
                missing_runs[runset]=run_list

In [118]:
for runset in [*missing_runs]:
     missing_runs[runset]=missing_runs[runset].tolist()

In [120]:
import yaml
with open('/scratch/rmueller/MIDOSS/Results/'+'missing_runs.yaml', 'w') as outfile:
    yaml.safe_dump(missing_runs, outfile)

In [108]:
missing_runs

{'near-BP_2000th-100_2021-10-02T144815': array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 13, 14, 17, 19, 20, 21, 23,
        24, 25, 26, 27, 29, 30, 35]),
 'near-BP_2004th-100_2021-10-02T144852': array([10, 15, 17, 19, 24]),
 'near-BP_46th-100_2021-10-02T142422': array([ 4, 21, 26, 28]),
 'near-BP_49th-100_2021-10-03T140133': array([42, 43, 52, 57, 59, 70, 72, 90, 91, 94, 95, 96, 97]),
 'near-BP_4th-100_2021-09-24T125848': array([92, 94, 96]),
 'near-BP_53th-100_2021-10-03T140217': array([ 6,  7,  8, 11, 12, 14, 16, 17, 19, 21, 23, 25, 26, 28, 29, 48, 49,
        55]),
 'near-BP_55th-100_2021-10-03T140235': array([24]),
 'near-BP_79th-100_2021-10-07T121854': array([ 9, 45, 50, 52, 54, 56, 75, 77, 78, 90]),
 'near-BP_88th-100_2021-10-11T190627': array([64, 70])}